In [ ]:
Retorno observado - ret
Volatilidade - vol
Max Drawdown - mdd
Meses observados - meses
Hit Ratio - hit_ratio
Sortino Ratio - sortino
Expected Shortfall - es
Calmar Ratio - calmar
Recovery Time - rec_time
Sharpe Ratio - sharpe
Information Ratio - info_ratio


In [19]:
def get_cdi_ferro_e_fogo():
    print("Baixando CDINI (Série 12) via JSON (Método Blindado)...")
    
    # MUDANÇA: Usamos JSON em vez de CSV. O JSON não bloqueia robôs.
    url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json"
    
    try:
        # verify=False pula a checagem de segurança do certificado do governo
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, verify=False, timeout=30)
        
        if response.status_code != 200:
            print(f"Erro na API: {response.status_code}")
            return pd.DataFrame()

        # O Pandas lê JSON nativamente
        df = pd.DataFrame(response.json())
        
        # [PERSONALIZAÇÃO] Se houver coluna __id, removemos
        if "__id" in df.columns:
            df = df.drop(columns=["__id"])
            
        # Tratamento de dados
        df.columns = ['data', 'valor']
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df['valor'] = pd.to_numeric(df['valor'], errors='coerce') # JSON já vem com ponto
        
        # Cálculo da Cota Acumulada
        df = df.dropna(subset=['valor']).sort_values('data')
        df['fator'] = 1 + (df['valor'] / 100)
        df['valor'] = df['fator'].cumprod()
        df['codigo'] = 'CDINI'
        
        print(f"  > Sucesso! {len(df)} registros baixados.")
        return df[['codigo', 'valor', 'data']]

    except Exception as e:
        print(f"  [ERRO CRÍTICO NO CDI]: {e}")
        return pd.DataFrame()
get_cdi_ferro_e_fogo()

Baixando CDINI (Série 12) via JSON (Método Blindado)...
Erro na API: 406


""


In [22]:
import pandas as pd
import yfinance as yf
import requests
from bcb import sgs
from datetime import datetime, date
from sqlalchemy import text
from common.postgresql import PostgresConnector as db

def get_robust_cdi(start_year=2000):
    print(f"Baixando CDI (Série 12) em blocos de 10 anos desde {start_year}...")
    current_year = datetime.now().year
    all_chunks = []
    
    # Divide a busca em janelas de 10 anos para evitar o erro do BCB
    for year in range(start_year, current_year + 1, 10):
        end_year = min(year + 9, current_year)
        start_str = f"{year}-01-01"
        end_str = f"{end_year}-12-31"
        
        try:
            # sgs.get já lida com os headers que causavam o erro 406
            df_chunk = sgs.get({'valor': 12}, start=start_str, end=end_str)
            if not df_chunk.empty:
                all_chunks.append(df_chunk)
        except Exception as e:
            print(f"  [AVISO] Falha no bloco {year}-{end_year}: {e}")

    if not all_chunks:
        return pd.DataFrame()

    df = pd.concat(all_chunks)
    
    # [PERSONALIZAÇÃO] Removendo __id se a biblioteca bcb a incluir futuramente
    if "__id" in df.columns:
        df = df.drop(columns=["__id"])

    # Ordenação e Cálculo do CDINI (Cota Acumulada)
    df = df.sort_index()
    df['valor'] = (1 + df['valor'] / 100).cumprod()
    df = df.reset_index().rename(columns={'Date': 'data'})
    df['codigo'] = 'CDINI'
    
    return df[['codigo', 'valor', 'data']]

def process_all_indices_v6():
    # 1. Mercado (Yahoo)
    try:
        print("Baixando Mercado (Yahoo Finance)...")
        tickers = ['^BVSP', 'BRL=X']
        data = yf.download(tickers, start='2000-01-01', progress=False)
        
        # Tratamento de MultiIndex para versões novas do yfinance
        if isinstance(data.columns, pd.MultiIndex):
            prices = data.xs('Adj Close', level=0, axis=1) if 'Adj Close' in data.columns.get_level_values(0) else data.xs('Close', level=0, axis=1)
        else:
            prices = data['Adj Close'] if 'Adj Close' in data.columns else data['Close']
            
        df_ibov = prices['^BVSP'].dropna().reset_index()
        df_ibov.columns = ['data', 'valor']
        df_ibov['codigo'] = 'IBOV'
        
        df_dolar = prices['BRL=X'].dropna().reset_index()
        df_dolar.columns = ['data', 'valor']
        df_dolar['codigo'] = 'DOLAR_VENDA'
        df_m = pd.concat([df_ibov, df_dolar])
    except Exception as e:
        print(f"Erro no Yahoo: {e}")
        df_m = pd.DataFrame()

    # 2. CDI (Obrigatório) - Método Robusto por Blocos
    df_c = get_robust_cdi(2000)
    if df_c.empty:
        raise Exception("O CDINI falhou em todas as tentativas. Processo interrompido.")

    # 3. IPCA (Série 433)
    print("Baixando IPCA...")
    try:
        # IPCA é mensal, então a restrição de 10 anos não se aplica (ou é menos rigorosa)
        df_ipca_m = sgs.get({'taxa': 433}, start='2000-01-01')
        
        # [PERSONALIZAÇÃO] Drop de __id
        if "__id" in df_ipca_m.columns:
            df_ipca_m = df_ipca_m.drop(columns=["__id"])
            
        # Projeção Diária do IPCA
        dr = pd.date_range(df_ipca_m.index.min(), datetime.now(), freq='D')
        df_ipca_d = pd.DataFrame({'data': dr}).set_index('data')
        df_ipca_d = df_ipca_d.join(df_ipca_m).ffill()
        df_ipca_d['valor'] = ((1 + df_ipca_d['taxa']/100)**(1/30)).cumprod()
        df_ipca_d['codigo'] = 'IPCADIANI'
        df_ipca_final = df_ipca_d.reset_index()[['codigo', 'valor', 'data']]
    except Exception as e:
        print(f"Erro no IPCA: {e}")
        df_ipca_final = pd.DataFrame()

    # Unificar
    df_final = pd.concat([df_m, df_c, df_ipca_final], ignore_index=True)
    
    # [PERSONALIZAÇÃO] Garantia final contra __id
    if "__id" in df_final.columns:
        df_final = df_final.drop(columns=["__id"])
        
    df_final['data'] = pd.to_datetime(df_final['data']).dt.date
    df_final.columns = [c.lower() for c in df_final.columns]

    # Salvar no Banco
    connector = db()
    with connector.engine.begin() as conn:
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS middle;"))
        df_final.to_sql('indices_cotas', conn, schema='middle', if_exists='replace', index=False)
        conn.execute(text("CREATE INDEX IF NOT EXISTS idx_indices_v6 ON middle.indices_cotas (data, codigo);"))
    
    print(f"Sucesso! Índices processados: {df_final['codigo'].unique()}")

if __name__ == "__main__":
    process_all_indices_v6()

Baixando Mercado (Yahoo Finance)...
Baixando CDI (Série 12) em blocos de 10 anos desde 2000...
Baixando IPCA...
Sucesso! Índices processados: ['IBOV' 'DOLAR_VENDA' 'CDINI' 'IPCADIANI']


In [29]:
query_cotas = """
    SELECT 
        cnpj_fundo, 
        COALESCE(id_subclasse, 'MASTER') as id_subclasse_clean,
        id_subclasse,
        dt_comptc, 
        vl_quota 
    FROM cvm.fi_doc_inf_diario_inf_diario_fi
    WHERE dt_comptc >= '2014-06-01'
"""
db = PostgresConnector()
df_cotas = db.read_sql(query_cotas)

In [30]:
df_cotas

""


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import text
from common.postgresql import PostgresConnector

def calculate_metrics_175_optimized():
    db = PostgresConnector()
    
    print("Carregando cotas (CVM 175 ready)...")
    # Coalesce no id_subclasse para evitar problemas de chave nula no DataFrame
    query_cotas = """
        SELECT 
            cnpj_fundo, 
            COALESCE(TEXT(id_subclasse), 'MASTER') as id_subclasse_clean,
            dt_comptc, 
            vl_quota 
        FROM cvm.cotas 
        WHERE dt_comptc >= '2014-06-01'
    """
    df_cotas = db.read_sql(query_cotas)
    
    # [PERSONALIZAÇÃO] Removendo __id conforme instrução
    if "__id" in df_cotas.columns:
        df_cotas = df_cotas.drop(columns=["__id"])

    # 1. TRATAMENTO DE DUPLICADOS (Prevenção do ValueError)
    # Mantemos apenas o último registro caso haja duplicidade para a mesma data/entidade
    print("Limpando duplicatas...")
    df_cotas = df_cotas.drop_duplicates(subset=['dt_comptc', 'cnpj_fundo', 'id_subclasse_clean'], keep='last')

    # Criar chave única para o pivot
    df_cotas['entity_id'] = df_cotas['cnpj_fundo'] + " | " + df_cotas['id_subclasse_clean']
    
    # Carregar Benchmarks
    df_bench = db.read_sql("SELECT data as dt_comptc, codigo, valor FROM middle.indices_cotas")
    bench_pivot = df_bench.pivot(index='dt_comptc', columns='codigo', values='valor').ffill()
    bench_ret = np.log(bench_pivot / bench_pivot.shift(1))

    print("Pivoteando matriz volumosa...")
    df_cotas['dt_comptc'] = pd.to_datetime(df_cotas['dt_comptc'])
    
    # Agora o pivot não falhará pois limpamos as duplicatas acima
    matrix = df_cotas.pivot(index='dt_comptc', columns='entity_id', values='vl_quota').sort_index().ffill()
    
    # Retornos logarítmicos matriciais
    returns = np.log(matrix / matrix.shift(1))
    
    # Fechamentos de mês (último dia útil)
    fechamentos = df_cotas['dt_comptc'].groupby([df_cotas['dt_comptc'].dt.year, df_cotas['dt_comptc'].dt.month]).max()
    fechamentos = fechamentos[(fechamentos >= '2015-01-01') & (fechamentos <= '2025-12-31')]

    janelas = {'6M': 126, '12M': 252, '24M': 504, '36M': 756, '48M': 1008, '60M': 1260}
    final_results = []

    # Cache de idade (primeira cota de cada entidade)
    primeira_cota = matrix.apply(lambda x: x.first_valid_index())

    for dt_ref in fechamentos:
        print(f"Calculando {dt_ref.date()}...")
        idx_fim = matrix.index.get_indexer([dt_ref], method='pad')[0]
        
        for label, dias in janelas.items():
            idx_ini = max(0, idx_fim - dias)
            if idx_fim <= idx_ini: continue
            
            window_ret = returns.iloc[idx_ini+1 : idx_fim+1]
            if window_ret.empty: continue

            # --- VETORIZAÇÃO ---
            cum_ret = np.exp(window_ret.sum()) - 1
            vol = window_ret.std() * np.sqrt(252)
            
            # Drawdown e MDD
            cum_prices = np.exp(window_ret.cumsum())
            running_max = cum_prices.cummax()
            mdd = ((cum_prices / running_max) - 1).min()
            
            # Sharpe (RF = CDINI)
            rf_total = np.exp(bench_ret['CDINI'].iloc[idx_ini+1 : idx_fim+1].sum()) - 1
            # Evita divisão por zero na volatilidade
            vol_safe = vol.replace(0, np.nan)
            sharpe = (cum_ret - rf_total) / vol_safe
            
            # Calmar e Sortino
            calmar = cum_ret / abs(mdd).replace(0, np.nan)
            downside_std = window_ret[window_ret < 0].std() * np.sqrt(252)
            sortino = (cum_ret - rf_total) / downside_std.replace(0, np.nan)
            
            # Expected Shortfall (5%)
            es = window_ret.quantile(0.05)

            # Idade e Filtro de Existência
            idade_real_meses = ((dt_ref - primeira_cota).dt.days / 30.44).round(2)
            
            batch = pd.DataFrame({
                'entity_id': matrix.columns,
                'dt_comptc': dt_ref.date(),
                'janela': label,
                'ret': cum_ret,
                'vol': vol,
                'mdd': mdd,
                'sharpe': sharpe,
                'sortino': sortino,
                'calmar': calmar,
                'es': es,
                'hit_ratio': (window_ret > 0).sum() / len(window_ret),
                'meses_observados': np.minimum(idade_real_meses, dias/21)
            }).dropna(subset=['ret'])

            batch = batch[batch['entity_id'].map(primeira_cota) <= dt_ref]
            
            # Reverter entity_id para colunas separadas
            split_cols = batch['entity_id'].str.split(" | ", expand=True, n=1)
            batch['cnpj_fundo'] = split_cols[0]
            batch['id_subclasse'] = split_cols[1].replace('MASTER', np.nan)
            
            final_results.append(batch.drop(columns=['entity_id']))

    # Consolidação e Information Ratio
    df_final = pd.concat(final_results, ignore_index=True)
    
    # [INFO RATIO]
    df_classes = db.read_sql("SELECT cnpj_fundo, classe FROM cvm.fi_cad_fi_hist_classe")
    # [PERSONALIZAÇÃO] Drop __id da tabela de classe
    if "__id" in df_classes.columns: df_classes = df_classes.drop(columns=["__id"])
    
    df_final = df_final.merge(df_classes.drop_duplicates('cnpj_fundo'), on='cnpj_fundo', how='left')
    
    # Benchmarks para IR
    ret_ibov = np.exp(bench_ret['IBOV'].sum())-1
    ir_ibov = (df_final['ret'] - ret_ibov) / df_final['vol'].replace(0, np.nan)
    ir_cdi = (df_final['ret'] - rf_total) / df_final['vol'].replace(0, np.nan)
    
    df_final['info_ratio'] = np.where(df_final['classe'].str.contains('Ações', na=False), ir_ibov, ir_cdi)

    print(f"Salvando {len(df_final)} linhas...")
    with db.engine.begin() as conn:
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS middle;"))
        df_final.to_sql('fundos_metricas_175', conn, schema='middle', if_exists='replace', index=False)
        conn.execute(text("""
            CREATE INDEX IF NOT EXISTS idx_metricas_175 
            ON middle.fundos_metricas_175 (cnpj_fundo, id_subclasse, dt_comptc, janela);
        """))

    print("Processo concluído com sucesso!")

if __name__ == "__main__":
    calculate_metrics_175_optimized()

Carregando cotas (CVM 175 ready)...
